## Recommender system

- In this part of the lab I willbuild a recommender system  for movies

In [2]:
# importing pandas and other packages needed in the assignment
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process


/Users/saragunnars/.local/share/virtualenvs/Machine-learning-Sara-Gunnars-bT69NVU4/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
#loading data files needed for the assignement
movies = '/Users/saragunnars/Skola/GitHub/Machine-learning-Sara-Gunnars/Data/movies.csv'
ratings = '/Users/saragunnars/Skola/GitHub/Machine-learning-Sara-Gunnars/Data/ratings.csv'

#also selecting which columns I want to use and assigning data type to the specifik cols
df_ratings = pd.read_csv(ratings, usecols= ['userId','movieId', 'rating'], dtype = {'userId':  'int32','movieId':'int32', 'rating':'float32'})
df_movies = pd.read_csv(movies, usecols= ['movieId', 'title'], dtype = {'movieId':'int32', 'title':'str'})

## Explorative data analysis

Here I will take a closer look at the dataset

In [4]:
df_movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [5]:
df_movies.info() # we have over 58 000 movies in the set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int32 
 1   title    58098 non-null  object
dtypes: int32(1), object(1)
memory usage: 681.0+ KB


In [6]:
df_ratings.head()

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In [7]:
# here we see the most frequent value in each column. 
# this means that userId 123100 has left the most ratings
# most revired movie is movieId 318
# Most common rating is 4
df_ratings.mode() 

,userId,movieId,rating
0,123100,318,4.0


### Top ten most rated movies

In [8]:
#creates a list with the 10 most rated movies
top_ten_list = df_ratings['movieId'].value_counts()[:10].index.to_list()
df_most_rated = pd.DataFrame(top_ten_list) #creating DF
df_most_rated = df_most_rated.rename(columns= {0 : 'movieID'}) #setting new column name
df_most_rated.index += 1 # setting index start count to 1
df_most_rated


,movieID
1,318
2,356
3,296
4,593
5,2571
6,260
7,480
8,527
9,110
10,1


In [9]:
#all the top ten most rated movies and their respective rating per review
top_ten_ratings = df_ratings[df_ratings["movieId"].isin([318, 356, 296, 593, 2571, 260, 480, 527, 110, 1])]
top_ten_ratings

,userId,movieId,rating
42,4,1,4.0
73,4,110,4.5
102,4,260,4.5
107,4,296,5.0
111,4,318,5.0
...,...,...,...
27753334,283228,260,5.0
27753337,283228,356,4.0
27753340,283228,480,4.5
27753343,283228,593,5.0


### Mean rating of top ten most rated movies

In [10]:
#The mean rating of the top ten most rated movies
top_ten_ratings['rating'].mean()

4.1022882

### Frequency of ratings

In [11]:
df_ratings['rating'].value_counts().to_frame() # shows how many times a rating has been given to a movie


,rating
4.0,7394710
3.0,5515668
5.0,4071135
3.5,3404360
4.5,2373550
2.0,1850627
2.5,1373419
1.0,886233
0.5,442388
1.5,441354


### Plot of the years and amount of movies represented in the dataset

In [12]:
df_movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [74]:
# looked at https://sparkbyexamples.com/pandas/pandas-split-column/ to reminde myself of how to split a DF column into new dataset
df_movies_years = df_movies['title'].str.split("(", expand=True)
df_movies_years[1] = df_movies_years[1].str.replace(")", '') #removed last ) in column one
#removes unnecessary column in DF
df_movies_years.drop(df_movies_years.columns[[2,3,4,5]], axis=1, inplace= True)
#Renames columns
df_movies_years.rename(columns = {0:'title', 1 :'year'}, inplace = True)

/var/folders/_j/wrygfb3x2znc30ht6mt37s0h0000gn/T/ipykernel_30530/4083348393.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_movies_years[1] = df_movies_years[1].str.replace(")", '') #removed last ) in column one


In [73]:
df_movies_years

,title,year
0,Toy Story,1995
1,Jumanji,1995
2,Grumpier Old Men,1995
3,Waiting to Exhale,1995
4,Father of the Bride Part II,1995
...,...,...
58093,The Great Glinka,1946
58094,Les tribulations d'une caissière,2011
58095,Her Name Was Mumu,2016
58096,Flora,2017
